## Implementação do Algoritmo Smith-Waterman
Fonte: https://github.com/slavianap/Smith-Waterman-Algorithm/blob/master/Script.py

In [ ]:
# Importa bibliotecas
import numpy as np

### Pontuações de match, mismatch e gaps

In [ ]:
# Pontuações
MATCH = +2
MISMATCH = -1
GAP = -2

### Valores constantes para mapear o traceback

In [ ]:
PARE = 0
ESQUERDA = 1
CIMA = 2
DIAGONAL = 3

## Lê um arquivo fasta com as sequências

In [ ]:
def leitor_fasta(arquivo_fasta):
    linhas = open(arquivo_fasta).readlines()
    nome_sequencia = linhas[0][1:]
    sequencia = linhas[1]
    return nome_sequencia.replace(" ", "").strip(), sequencia.strip()

seq1 = leitor_fasta("fasta3.fasta")
print(seq1[0])
print(seq1[1])

seq2 = leitor_fasta("fasta4.fasta")
print(seq2[0])
print(seq2[1])

Seq1[organism=Carpodacusmexicanus][clone=6b]actin(act)mRNA,partialcds
CCTTTATCTAATCTTTGGAGCATGAGCTGGCATAGTTGGAACCGCCCTCAGCCTCCTCATCCGTGCAGAATAATAATTTTCTTTATAGTAATACCAATCATGATCGGTGGTTTCGGAAACTGACTAGTCCCACTCATAAT
Seq2[organism=unculturedbacillussp.][isolate=A2]corticotropin(CT)gene,completecds
GGTAGGTACCGCCCTAAGNCTCCTAATCCGAGCAGAACTANGCCAACCCGGAGCCCTTCTGGGAGACGACTCAACACCACCTTCTTTGACCCAGCAGGAGGAGGAGACCCAGTACTATACCAGCACCTATTCTGATTCTT


### Cria matrizes para armazenar as pontuações e o traceback

In [ ]:
matriz_principal = np.zeros(shape=(len(seq1[1])+1,len(seq2[1])+1))
matriz_traceback = np.zeros(shape=(len(seq1[1])+1,len(seq2[1])+1))
print(matriz_principal)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Variáveis para encontrar a célula com maior pontuação
É dessa célula que inicia o traceback

In [ ]:
pontuacao_maxima = -1
indice_maximo = (-1, -1)

### Preenche a matriz principal com as pontuações
Já marcando a célula anterior que deu origem ao valor de uma célula

In [ ]:
# Percorre toda a matriz
for i in range(1,len(seq1[1])+1):
  for j in range(1,len(seq2[1])+1):

    # Pontuação da diagonal (match/mismatch)
    if(seq1[1][i-1] == seq2[1][j-1]):
      pontuacao_diagonal = matriz_principal[i - 1,j - 1] + MATCH
    else:
      pontuacao_diagonal = matriz_principal[i - 1, j - 1] + MISMATCH

    # Pontuação da vertical (gap)
    pontuacao_vertical = matriz_principal[i-1,j] + GAP

    # Pontuação da horizontal (gap)
    pontuacao_horizontal = matriz_principal[i,j-1] + GAP

    # Preenche a matriz com o maior valor (substituimos os negativos por 0)
    matriz_principal[i,j] = max(0,pontuacao_diagonal,pontuacao_vertical,pontuacao_horizontal)

    # Já armazena a célula anterior que deu origem ao valor da célula atual (trace)
    if matriz_principal[i,j] == 0: # Não aponta pra nenhuma célula
      matriz_traceback[i,j] = PARE

    if matriz_principal[i,j] == pontuacao_horizontal: # Aponta para a célula da esquerda
      matriz_traceback[i,j] = ESQUERDA

    if matriz_principal[i,j] == pontuacao_vertical: # Aponta para a célula de cima
      matriz_traceback[i,j] = CIMA

    if matriz_principal[i,j] == pontuacao_diagonal: # Aponta para a célula da diagonal
      matriz_traceback[i,j] = DIAGONAL

    # Já verifica a célula com a máxima pontuação na matriz
    if matriz_principal[i,j] >= pontuacao_maxima:
      indice_maximo = (i,j)
      pontuacao_maxima = matriz_principal[i,j]

print(matriz_principal)
print(matriz_traceback)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  2.  0.  0.]
 [ 0.  0.  0. ...  2.  1.  0.]
 ...
 [ 0.  0.  0. ... 58. 56. 54.]
 [ 0.  0.  0. ... 59. 57. 55.]
 [ 0.  0.  0. ... 60. 61. 59.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 3. 1. 0.]
 [0. 0. 0. ... 3. 3. 0.]
 ...
 [0. 0. 0. ... 3. 3. 3.]
 [0. 0. 0. ... 3. 3. 3.]
 [0. 0. 0. ... 1. 3. 3.]]


### Inicia o traceback

In [ ]:
# Inicialização de variáveis
seq1_alinhada = ""
seq2_alinhada = ""
alinhamento_atual_seq1 = ""
alinhamento_atual_seq2 = ""
(max_i, max_j) = indice_maximo

# Faz o traceback e monta o melhor alinhamento local
while matriz_traceback[max_i,max_j] != PARE:

  if matriz_traceback[max_i,max_j] == DIAGONAL: # Está apontando para diagonal
    alinhamento_atual_seq1 = seq1[1][max_i-1]
    alinhamento_atual_seq2 = seq2[1][max_j-1]
    max_i = max_i - 1
    max_j = max_j - 1

  elif matriz_traceback[max_i,max_j] == CIMA: # Está apontando para cima
    alinhamento_atual_seq1 = seq1[1][max_i-1]
    alinhamento_atual_seq2 = '-'
    max_i = max_i - 1

  elif matriz_traceback[max_i,max_j] == ESQUERDA: # Está apontando para a esquerda
    alinhamento_atual_seq1 = '-'
    alinhamento_atual_seq2 = seq2[1][max_j-1]
    max_j = max_j - 1

  # Vai montando o alinhamento
  seq1_alinhada = alinhamento_atual_seq1 + seq1_alinhada
  seq2_alinhada = alinhamento_atual_seq2 + seq2_alinhada

### Mostra o alinhamento local

In [ ]:
print(seq1_alinhada)
print(seq2_alinhada)

GTTGGAACCGCCCTCAGCCTCCTCATCCGTGCAGAA-T-----AA----TA-ATTTTCT-TTATA-G--T-AATACCA--ATC-ATGA-TC-GGTGGTTTCGGA-A-ACTG-ACTAGTCCCA-CTCATAAT
GTAGGTACCGCCCTAAGNCTCCTAATCCGAGCAGAACTANGCCAACCCGGAGCCCTTCTGGGAGACGACTCAACACCACCTTCTTTGACCCAGCAGGAGGAGGAGACCCAGTACTA-TACCAGCACCTATT
